<img src="https://www.ceste.es/wp-content/uploads/2022/03/logotipo-ceste-escuela-internacional-de-negocios.svg" width="500"/>

# Procesamiento de lenguaje natural

## Analisis Sentimiento Imdb con Bert (embeddings y modelo)

Pablo Gómez Guerrero

## Análisis de sentimiento

En esta ocasión trataremos el mismo reto que vimos anteriormente sobre la base de datos de opioniones de IMDB, con la intención de mejorar las métricas que teníamos (*accuracy 0.87 aprox*)

Solo por recordar, en el [reto](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) tenemos 50k opiniones de películas etiquetadas con un sentimiento positivo o negativo.

## BERT

En este notebook usaremos BERT (Bidirectional Encoder Representations from Transformers) que es un Transformer desarrollado por Jacob Devlin, Ming-Wei Chang, Kenton Lee and Kristina Toutanova en 2018 para Google. BERT está entrenado con unos 2500 Millones de palabras en inglés obtenidas de Wikipedia y 800 Millones de palabras de BookCorpus.

BERT es famoso por su potente rendimiento para tareas de NLP siendo el estado del arte en 2018. Una de las tareas que mejor realiza es el análisis de sentimiento, como veremos a continuación haciendo lo que se llama un Fine-tuning.

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CESTE_2023/data/IMDB Dataset.csv")
df.sample()

In [ ]:
#  clasificador y Tokenizer de BERT
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model.summary()

In [ ]:
# Vamos a cambiar la anotaciones

def cat2num(value):
    if value=='positive':
        return 1
    else:
        return 0


In [ ]:
# Dividimos el dataset en train y test

df['sentiment']  =  df['sentiment'].apply(cat2num)
train = df[:45000]
test = df[45000:]

### Procesamiento de datos

Para hacer el Fine-tuning con BERT, como en otras ocasiones, hay varios pasos que necesitamos dar previamente.


1.   tokenizar y separar las frases para clasificar
2.   Las secuencias tienen que tener una logitud igual - padding
3.   Crear una capa que se llama mascara de atención (para el transformer) con un array de 0s (pad token) y uno de 1s (real token)



In [ ]:
# Veamos primero como es el tokenizador

example='We will work on sentiment analysis using BERT in this notebook'
tokens = tokenizer.tokenize(example)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(token_ids)

## Particularidades de BERT

Bert trabaja con un vocabulario fijo de ~30k palabras, divididos subpalabras y caracteres. Así pues, no todas las palabras del diccionario están representadas en el vocabulario, para ello se usan las subpalabras y caracteres.

El tokenizador juega un papel fundamental pues decide si mantener las palabras o dividirlas en algo más pequeño.

### Representaciones especiales

Además BERT necesita de unas representaciones especiales para trabajar con el texto. Estas respresentaciones son marcas dentro del texto que ayudan al sistema.

1.   [SEP] - marca el fin de frase
2.   [CLS] - marca el inicio de frase
3.   [PAD] - padding
4.   [UNK] - tokens que BERT no entiende porque no están en el set de entrenamiento

## Pasos para entrenar

Ahora vamos a adaptar los dataset a la forma con la que trabaja BERT. Para ellos usaremos 2 pasos.

1. Convertir los datos en muestras válidas
2. Converitr las muestras en objetos tokenizados



### convert_data_to_examples

Esta función convertirá nuestros datsets de entrenamiento y validación en ejemplos de entrada para BERT. Usamos unas labda para recorrer todos las muestras.



In [ ]:
def convert_data_to_examples(train, test, review, sentiment):
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # No definimos este parámetro pero si lo hiciéramos es como un marcador que nos puede servir para guardar esta muestra para el futuro
                                                          text_a = x[review],
                                                          label = x[sentiment]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                          text_a = x[review],
                                                          label = x[sentiment]), axis = 1,)

    return train_InputExamples, validation_InputExamples

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train,  test, 'review',  'sentiment')

In [ ]:
train_InputExamples[0]

### convert_examples_to_tf_dataset

Esta es la parte más compleja del notebook. Vamos a transformar los Ejemplos de entrada anteriormente definidos en Features que entiende BERT. Se pueden ver 2 pasos fundamentales.

1. Encoding con el tokenizador. Aquí se configuran la forma de los tokens de cada frase
2. Cómo añade los tokens, la capa de atención y tipo de token dentro de un objeto y fuera la etiqueta de la frase.

Esta manipulación que hemos hecho es lo que nos da el poder de clasificar nuestro dataset usando BERT.

In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> InputFeatures

    for e in tqdm(examples):
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens = True,    # añadimos 'CLS'  'SEP'
            max_length=max_length,    # truncamos if len(s) > max_length
            return_token_type_ids = True,
            return_attention_mask = True,
            pad_to_max_length = True, # pads to the right by default
            truncation = True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(InputFeatures( input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label) )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'review'
LABEL_COLUMN = 'sentiment'

In [ ]:
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2) # Desordenamos los ejemplos y los guardamos en batch de 32 elementos

In [ ]:
validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

### Listos para entrenar

Ya hemos preparado el set de entrenamiento y validación, vamos a entrenar el modelo.

**ATENCIÓN: Nos tomará 1 hora o más.**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

In [ ]:
pred_sentences = ['Very boring movie, I would not recommend it to my friends', 'Wow, blew my mind, the animation and the story are amazing', 'Thoughtful and complex story but I enjoyed it very much']

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')   # Tokenizamos antes de enviar al modelo
tf_outputs = model(tf_batch)

# Softmax para la clasificación final porque el modelo devuelve otra cosa (logits) y luego el índice mayor es el que  nos ayuda a decidir si es positivo o negativo

tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
    print(pred_sentences[i], ": ", labels[label[i]])

## Lecturas interesantes sobre BERT:
1. https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671
2. https://medium.com/@dhartidhami/understanding-bert-word-embeddings-7dc4d2ea54ca